In [2]:
import numpy as np

# Q1

In [6]:
def resolutionSystem1Euler(alpha,beta,delta,gamma,y0,z0,t0,dt,N):
    "Cette fonctipn permet de resoudre le système (1)"
    temps = np.arange(0,N)*dt+t0 # si j'ai bien compris la question (dt=10/(N-1) pour 10 années)
    y = [y0]
    z = [z0]
    for i in range(N-1):
        y.append(y[i]+dt*y[i]*(alpha-beta*z[i]))
        z.append(z[i]+dt*z[i]*(delta*y[i]-gamma))

    return temps,y,z

In [11]:
def h(y,z,alpha,beta,delta,gamma) :

    return y*(alpha-beta*z),z*(delta*y-gamma)

In [13]:
def resolutionSystem1EulerRK4(alpha,beta,delta,gamma,y0,z0,t0,dt,N):
    "Cette fonctipn permet de resoudre le système (1)"
    temps = np.arange(0,N)*dt+t0 # si j'ai bien compris la question (dt=10/(N-1) pour 10 années)
    y = [y0]
    z = [z0]
    for i in range(N-1):
        k1 = h(y[i],z[i],alpha,beta,delta,gamma)
        k2 = h(y[i]+0.5*dt*k1[0],z[i]+0.5*dt*k1[1],alpha,beta,delta,gamma)
        k3 = h(y[i]+0.5*dt*k2[0],z[i]+0.5*dt*k2[1],alpha,beta,delta,gamma)
        k4 = h(y[i]+dt*k3[0],z[i]+dt*k3[1],alpha,beta,delta,gamma)
        y.append(y[i]+(dt/6)*(k1[0]+2*k2[0]+2*k3[0]+k4[0]))
        z.append(z[i]+(dt/6)*(k1[1]+2*k2[1]+2*k3[1]+k4[1]))

    return temps,y,z

# Q2

In [4]:
Nsamples = 1000
alpha = np.random.lognormal(0.5,0.2,Nsamples)
beta = np.random.lognormal(0.05,0.02,Nsamples)
delta = np.random.lognormal(0.05,0.02,Nsamples)
gamma = np.random.lognormal(0.5,0.2,Nsamples)
y0 = np.random.lognormal(1,1,Nsamples)
z0 = np.random.lognormal(1,1,Nsamples)

In [22]:
N = 1001
dt = 20/(N-1)
t0 = 1900
solution = resolutionSystem1EulerRK4(alpha,beta,delta,gamma,y0,z0,t0,dt,N)

In [ ]:
def MC(samples):

    mean = np.mean(samples) 
    sigma2 = np.mean(samples**2)-mean**2
    return mean, sigma2

### t0=1900

In [24]:
meany,sigma2y = MC(y0)
meanz,sigma2z = MC(z0)
# tracer cette loi de lognormal

### t1=1901

### t2=1920